In [24]:
import pygame
import random
import math

sound1 = 'ding.mp3'
imgelv = "elv.png"
imgfloor = "b.png"
imgpassenger = "men1.png"

class building:
    numfloors = 18

    def __init__(self, screen, floorheight, floorwidth, numofelv=2, space=1):
        self.screen = screen
        self.elevators = [elevator(screen, 250 + j * 80, floorheight) for j in range(numofelv)]
        self.floors = [floor(screen, 0, (building.numfloors - 1 - i) * floorheight, floorheight, floorwidth, i, self) for i in range(building.numfloors)]


    def update(self, floor):
        selectedelevator = self.elevators[0]
        mindistance = abs(selectedelevator.myfloor - floor.floornum)
        for elevator in self.elevators:
            if elevator.ismovingindirection(floor.floornum, elevator.setdirection(floor.floornum)):
                distance = abs(elevator.myfloor - floor.floornum)
                if distance < mindistance:
                    mindistance = distance
                    selectedelevator = elevator
        selectedelevator.addtarget(floor)
        selectedelevator.moving=True
    def update1(self,dt):
        for i in self.elevators:
            
            if i.moving==True:
              i.move(dt)
            i.draw()

    def draw(self):
        for floor in self.floors:
            floor.draw()
        for elevator in self.elevators:
            elevator.draw()

class elevator:
    def __init__(self, screen, initialposx, floorheight, step=10):
        self.screen = screen
        self.ypos = 0
        self.xpos = initialposx
        self.floorheight = floorheight
        self.step = floorheight / 100
        self.myfloor = self.ypos // self.floorheight
        self.direction = None
        self.targets = []
        self.passengers = []
        pygame.mixer.init()
        self.dingsound = pygame.mixer.Sound(sound1)
        self.moving = False

        originalimage = pygame.image.load(imgelv).convert_alpha()
        self.image = pygame.transform.scale(originalimage, (75, int(floorheight * 0.75)))
        self.rect = self.image.get_rect(topleft=(self.xpos, self.ypos))

    def addtarget(self, floor):
        if floor not in self.targets:
            self.targets.append(floor)
            self.targets.sort(key=lambda x: x.floornum, reverse=self.direction == -1)
        if self.direction is None:
            self.setdirection(floor.floornum)

    def setdirection(self, floornum):
        if self.myfloor < floornum:
            self.direction = 1
        elif self.myfloor > floornum:
            self.direction = -1

    def move(self,dt):
       
       
       
        
        if not self.targets:
            return
        if self.direction is None:
            print("Direction not set, cannot move.")
            return
        a3=500/dt
        self.rect.y += (self.direction)*a3
        
        self.myfloor = self.rect.y / self.floorheight
        self.myfloor=math.floor(self.myfloor)
        if self.direction<0:
           self.myfloor = (self.rect.y // self.floorheight)
        targetfloor = self.targets[0]
        if self.myfloor == targetfloor.floornum:
            self.dingsound.play()
            self.moving=False
            # targetfloor.loadpassengers(self)
            self.targets.pop(0)
            if not self.targets:
                self.direction = None
        pygame.time.set_timer(pygame.USEREVENT, 100)

    def ismovingindirection(self, floornum, direction):
        if self.direction is None:
            return True
        return (self.direction == 1 and self.myfloor <= floornum) or (self.direction == -1 and self.myfloor >= floornum)

    def draw(self):
        self.screen.blit(self.image, self.rect)

        font = pygame.font.Font(None, 36)  

        text_surface = font.render(str(self.rect.y), True, (0, 0, 0))  
        text_rect = text_surface.get_rect(center=(self.rect.centerx, self.rect.y + 20))  

        # self.screen.blit(text_surface, text_rect)

        # button_text = font.render(str(self.myfloor), True, (255, 255, 255))  
        # button_rect_width = button_text.get_width() + 10  
        # button_rect_height = button_text.get_height() + 10
        # button_rect = pygame.Rect(self.rect.x + 10, self.rect.bottom - button_rect_height - 10, button_rect_width, button_rect_height)  

        # pygame.draw.rect(self.screen, (0, 128, 0), button_rect)  

        # button_text_rect = button_text.get_rect(center=button_rect.center)
        # self.screen.blit(button_text, button_text_rect)

            

class floor:
    def __init__(self, screen, posx, posy, height, width, floornum, building):
        self.screen = screen
        self.posx = posx
        self.posy = posy
        self.width = width
        self.height = height
        self.floornum = floornum
        self.building = building
        
        
        brickimage = pygame.image.load(imgfloor).convert_alpha()
        self.bricktexture = pygame.transform.scale(brickimage, (width, height))
        self.rect = self.bricktexture.get_rect(topleft=(posx, posy))
        self.buttonrect = pygame.Rect(posx + width - 100, posy + 10, 90, 30)

        
    def draw(self):
        self.screen.blit(self.bricktexture, self.rect)
        font = pygame.font.Font(None, 36)
        text_surface = font.render(str(self.floornum), True, (0, 0, 0))
        text_rect = text_surface.get_rect(center=(self.posx + self.width // 2, self.posy + self.height // 2))
        self.screen.blit(text_surface, text_rect)
        pygame.draw.rect(self.screen, (0, 128, 0), self.buttonrect)
        button_text = font.render(str(self.floornum), True, (255, 255, 255))
        button_text_rect = button_text.get_rect(center=self.buttonrect.center)
        self.screen.blit(button_text, button_text_rect)
      

    def checkclick(self, position):
        if self.buttonrect.collidepoint(position):
            self.callelevator()

    def callelevator(self):
        
        self.building.update(self)



def run_game():
    screen_width, screen_height, floor_height, floor_width, num_elevators=800, 600, 70, 200, 5
    pygame.init()
    screen = pygame.display.set_mode((1800, 1000))
    pygame.display.set_caption('Building Simulation')
    clock = pygame.time.Clock()
    building_instance = building(screen, floor_height, floor_width, num_elevators) 
    a=1
    running = True
    while running:
        if a==1:
         dt = clock.tick(60)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                print("Mouse Clicked at:", event.pos)  
                if event.button == 1:  
                    for floor in building_instance.floors:
                        floor.checkclick(event.pos)

        screen.fill((255, 255, 255))  
        building_instance.draw()  
        building_instance.update1(dt)
        pygame.display.flip()  
        clock.tick(60)  

    pygame.quit()


run_game()

Mouse Clicked at: (160, 511)
